# Week 12 Problem Set

## Cohort Sessions

**CS1.** Define an Abstract Class for a State Machine, called `StateMachine`. The class has two properties:
- `state`: which is the current state of the machine
- `start_state`: which is the initial state of the machine

The class should define the following methods:
- `start()`: this method set the `state` property using the value in `start_state`. Once `state` has a value, the machine is considered started.
- `step(inp)`: this method takes in the current input and returns the current output. This method should move the state machine to the next state based on the current input and its current state. You should call `get_next_values(state, inp)` in your implementation.
- `done(state)`: this method always return `False`. A child class can override thid method to give a different condition to end the state machine.
- `is_done()`: is to be used internally to check if the state machine should terminate or not. This method simply calls `done(state)` and pass on the current `state`. The method `transduce(inp_list)` calls this method to check if it should terminates or not.
- `transduce(inp_list)`: this method calls `start()` to initialize the `state` with the `start_state` and run the state machine by calling `step(inp)` for every item in the `inp_list`. The method runs the state machine and produces the output list according to the number of input in the `inp_list` or when the state machine terminates according to the output of `is_done()` method. This method should call `is_done()` to see if it should terminate at a particular state.

This class should be an Abstract Class. Implement the following way:
- `SM` class inherits from `abc.ABC`, which is Python's Abstract Base Class (ABC). 
- Any implementation of State Machine's instances must declare the following property: `start_state`.
- Any implementation of State Machine's instances must implement the following abstract method: `get_next_values()` that takes in the current `state` and the the current `input` and output a tuple of the `next_state` and the current `output` . 


In [13]:
from abc import ABC, abstractmethod

class StateMachine(ABC):
    
    def start(self):
        self.state = self.start_state

    def step(self, inp):
        ns, o = self.get_next_values(self.state, inp)
        self.state = ns
        return o
        
    def transduce(self, inp_list):
        output = []
        self.start()
        for inp in inp_list:
            if not self.is_done():
                o = self.step(inp)
                output.append(o)
        return output
        
    @abstractmethod
    def get_next_values(self, state, inp):
        if state == "off":
            if inp == 1:
                next_state = "on"
            else:
                next_state = "off"
        elif state == "on":
            if inp == 1:
                next_state = "off"
            else:
                next_state = "on"
        output = next_state
        return next_state, output

    def done(self, state):
        return False

    def is_done(self):
        return self.done(self.state)

    

In [14]:
class Test(StateMachine):
    start_state = 0

    def get_next_values(self, state, inp):
        next_state = state + inp
        output = next_state
        return next_state, output

    def done(self, state):
        if state == -1:
            return True
        else:
            return False

class NoImplement(StateMachine):
    def __init__(self):
        start_state = 0
    
t1 = Test()
t1.start()
assert t1.state == 0
out = t1.step(2)
assert t1.state ==2 and out == 2

t2 = Test()
out = t2.transduce([1,2,3,4])
assert out == [1, 3, 6, 10]

t3 = Test()
out = t3.transduce([1, -2, 3])
assert out == [1, -1]

try:
    t4 = NoImplement()
    raise AssertionError
except TypeError:
    pass

In [15]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**CS2.** *Coke Machine:* In this problem, you will implement in Python the behavior of a simplified coke-dispensing machine. The behavior of such a machine is captured in the state diagram shown in the Figure below. The machine consists of two states labelled 0 and 1. The state diagram does not show what the machine would do if an unexpected coin is inserted. Therefore, assume that any unexpected coin is returned to the user without a change in the machines state. Thus, on your own, you may want to complete the Figure below to add in the missing transitions.

![](https://www.dropbox.com/s/kzk6nkdss7wvw85/coke_sm.png?raw=1)

Each directed arc in the state diagram is labelled as $x/y$ where $x$ denotes the input received and $y$, the output generated. For example, the arc that connects state 0 to state 1 that’s labelled `50/(50, ’--’,0)` means that when the dispenser receives 50¢ (50 before the /) in state 0, it moves to state 1 and generates an output of `(50, ’--’,0)`. This tuple of values in the output indicates that the dispenser display shows 50 which is the amount deposited by the user, no coke has been dispensed yet as indicated by `--`, and no change has been returned to the user as indicated by the last entry which is a 0.
The machine accepts only 50¢ and one dollar (100¢) coins. It has a display that shows how many cents have been deposited.
- State 0: When a 50¢ coin is deposited the dispenser moves to state 1. At this moment in time, the display shows 50 but nothing is dispensed and no change is returned. If a dollar coin is deposited, the machine continues to display 0, dispenses coke, and does not return any money (well, why should it!).
- State 1: When a 50¢ coin is deposited the dispenser moves to state 0. At this moment in time, the display shows 0, coke is dispensed and no change is returned. If a dollar coin is deposited the machine continues to display 0, dispenses coke, and returns 50¢.

We wish to write a program that simulates the behavior of the coke dispenser as described above. We will write a class named CokeMachine that contains properties and methods as described below:
- `CokeMachine` class is a subclass of `StateMachine` class.
- `CokeMachine` class has a class attribute called start_state which is the starting state of the machine. This attribute should be initialized to 0, which represents state 0 in the diagram above.
- `CokeMachine` class implements the abstract method `get_next_values(state, inp)` that takes in the current state and the input, and returns the next state and output as a tuple. Think about the following: which state represents the following scenarios?
    - the coke machine is waiting for a valid coin to be deposited 
    - the coke machine has a 50-cent coin in it

Sample Interaction:
```
cm = CokeMachine()
cm.start()
print(c.step(50))
print(c.step(50))
print(c.step(100))
print(c.step(10))
print(c.step(50))
print(c.step(100))
print(c.step(10))
```

The output should be:
```
(50, "--", 0)
(0, "coke", 0)
(0, "coke", 0)
(0, "--", 10)
(50, "--", 0)
(0, "coke", 50)
(0, "--", 10)
``` 

In [16]:
class CokeMachine(StateMachine):

    def __init__(self):
        self.start_state = 0

    def get_next_values(self, state, inp):
        if state == 0 and inp == 50:
            next_state = 1
            output = (50,"--",0)
        elif state == 0 and inp == 100:
            next_state = 0
            output = (0,"coke",0)
        elif state == 1 and inp == 50:
            next_state = 0
            output =  (0,"coke",0)
        elif state == 1 and inp == 100:
            next_state = 0
            output = (0,"coke",50)
        else:
            next_state = state
            output = (0 if state == 0 else 50, "--", inp)
            
        return next_state, output

In [17]:
cm = CokeMachine()
cm.start()
assert cm.state == 0
out = cm.step(50)
assert out == (50, "--", 0) and cm.state == 1
out = cm.transduce([50, 50, 100, 10, 50, 100, 10])
assert out == [(50, '--', 0), (0, 'coke', 0), (0, 'coke', 0), (0, '--', 10), (50, '--', 0), (0, 'coke', 50), (0, '--', 10)]

In [18]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**CS3.** *Simple Account:* In this problem, you will need to create a state machine that simulates a simple bank account. This is similar to the Accumulator state machine in the notes. The only difference is that any withdrawal when the balance is less than \\$100  incurs a \\$5 charge. The state machine should fulfill the following:
- The starting balance is specified when instantiating the object.
- The output of the state machine is the current balance after the transaction.

Sample interaction:
```
acct = SimpleAccount(110)
acct.start()
print(acct.step(10))
print(acct.step(-25))
print(acct.step(-10))
print(acct.step(-5))
print(acct.step(20))
print(acct.step(20))
```

The expected output is:
```
120
95
80
70
90
110
```

In [19]:
class SimpleAccount(StateMachine):
    def __init__(self, balance):
        self.start_state = balance
    def get_next_values(self, state, inp):
        next_state = state + inp - 5 if state < 100 and inp < 0 else state + inp
        output = next_state
    
        
        
        return next_state, output

In [20]:
acct = SimpleAccount(110)
out = acct.transduce([10, -25, -10, -5, 20, 20])
assert out == [120, 95, 80, 70, 90, 110]

In [21]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**CS4.** *Graph:* We can consider graph search problem as a state-space search, where each state is a node in the graph and the transition between one state to another as an edge in the graph. Recall that we can represent graph using either a Dictionary or a Class. In this problem, we will use Dictionary to represent a graph where each node represents a state. For example, the image below can be represented as follows:

```
map = {"S": ["A", "B"],
        "A": ["S", "C", "D"],
        "B": ["S", "D", "E"],
        "C": ["A", "F"],
        "D": ["A", "B", "F", "H"],
        "E": ["B", "H"],
        "F": ["C", "D", "G"],
        "H": ["D", "E", "G"],
        "G": ["F", "H"]}
```

![](https://www.dropbox.com/s/3q0hsnbgssl2saa/state_space_map.png?raw=1)

Let's consider an action to be the index of the list in that Dictionary. For example, if my current state is "S", action 0 will give "A" as the next state while action 1 will give "B" as the next state. 

Write `MapSM` class and override the `get_next_values()` method to 
- `__init__(start)`: which initialize the `start_state` with `start` during object instantiation.
- `get_next_values(state, inp)`: this method produces the next state based on the input and the current state. The next states are the neighbours of the current vertex based on the `inp`. The `inp` argument is the index of the next vertex to be visited. The state machine should remain in the current state if the input is not valid or when the current vertex has no neighbours.

The class `MapSM` should implement `StateSpaceSearch` class which is an abstract class inheriting from `StateMachine` abstract class. The `StateSpaceSearch` abstract class requires `MapSM` to implement the properties:
- `statemap`: which gives the dictionary or the map of the state space.
- `legal_inputs`: which gives a `set` of legal inputs of this state space. This is a set of integers from 0 up to $n-1$ where $n$ is the largest number of neighbours of a state in the graph. 


In [23]:
from abc import abstractmethod

class StateSpaceSearch(StateMachine):
    @property
    @abstractmethod
    def statemap(self):
        pass

    @property
    @abstractmethod
    def legal_inputs(self):
        pass

    

In [24]:
class MapSM(StateSpaceSearch):
        
    def __init__(self, start):
        self.start_state = start

    @property
    def statemap(self):
        statemap = {"S": ["A", "B"],
                    "A": ["S", "C", "D"],
                    "B": ["S", "D", "E"],
                    "C": ["A", "F"],
                    "D": ["A", "B", "F", "H"],
                    "E": ["B", "H"],
                    "F": ["C", "D", "G"],
                    "H": ["D", "E", "G"],
                    "G": ["F", "H"]}
        return statemap

#     @property
#     def legal_inputs(self):
#         nums = []
#         for i in self.statemap:
#             nums.append(len(self.statemap[i]))
#         n = max(nums)
#         return (set(range(0,n)))

  
#     def get_next_values(self, state, inp):
#         if state in self.statemap and inp < len(self.statemap[state]):
#             next_state = self.statemap[state][inp]
#         else:
#             next_state = state
#         output = next_state
#         return next_state, output
    @property
    def legal_inputs(self):
        max_neighbour = -1
        for state, neighbours in self.statemap.items():
            if max_neighbour < len(neighbours):
                max_neighbour = len(neighbours)
        
        # max_neighbour is just an integer
        # we need to return a set 
        return set(range(0, max_neighbour))

    # eg if state == S, and inp == 1, 
    # return "B", "B"
    def get_next_values(self, state, inp):
        # use a method that will not result in an error if the key isn't found
        neighbours = self.statemap.get(state, None)
        # why not: neighbours = self.statemap[state]?
        # because if state is not a valid key, the above will return error

        # default values
        next_state = state
        output = state 

        # if neighbours is None:
        #     return next_state, output
        
        if neighbours != None and inp < len(neighbours):
            next_state = neighbours[inp]
            output = next_state
        
        return next_state, output
            
    

In [25]:
mapSM = MapSM("S")
mapSM.start()
ans = mapSM.transduce([0, 1, 1, 2, 0])
assert ans == ["A", "C", "F", "G", "F"]
assert mapSM.legal_inputs == set(range(4))

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**CS5.** Create a class to represent a node in a state-space search called `SearchNode`. The object instance of `SearchNode` has the following properties which should be initialized during instantiation.
- `state`: which is the state or the label of the node.
- `action`: which is the action that was taken to arrive at the node.
- `parent`: which is the parent search node from which the current search node can be reached. If a node has no parent, this property should be initialized to `None`.

The class has the following method:
- `path()`: which returns a list of `Step` from the root node to the current node. The `Step` object is a class and is defined in the template. A `Step` object has two properties: `action` and `state`. You should use recursion for this method `path()`. The base case is when the node has no parent. In this case the solution contains only one step which is the action and the state of the current node. The recursive case is when the current has a `parent` object. In this case, you have to traverse to the ancestors' node until it reach a node which has no `parent` object. 
- `in_path(state)`: which returns `True` if the state in the argument is in the path. *Hint: use recursion and the parent's `in_path()` method.*

In [24]:
class Step:
    def __init__(self, action, state):
        self.action = action
        self.state = state
    
    def __eq__(self, other):
        return self.action == other.action and self.state == other.state
  
    def __str__(self):
        return f"action: {self.action:}, state: {self.state:}"

In [19]:
class SearchNode:
    def __init__(self, action, state, parent):
        self.action = action
        self.state = state
        self.parent = parent
  
    # @return
    # list of Step instances
    # Example: S -> A -> C
    # C.path() SHOULD return [Step(None, S), Step(ActionA, A), Step(ActionC, C)]
    # C.path() go to else clause:
        # return A.path() + [Step(ActionC, C)]
    # Requires us to compute A.path() before C.path() can return
    # A.path() go to else clause:
        # return S.path() + [Step(ActionA, A)]
    # Requires us to compute S.path() before A.path() can return
    # S.path() go to if clause:
        # return [Step(None, S)] to A.path()
    # A.path() now executes:
        # return [Step(None, S)] + [Step(ActionA, A)] to C.path()
    # C.path() now executes:
        # return [Step(None, S)] + [Step(ActionA, A)] + [Step(ActionC, C)] 

    def path(self):
        # base state
        if self.parent is None:
            return [Step(self.action, self.state)]
        else:
            return self.parent.path() + [Step(self.action, self.state)]
        
    # @args
    # state --> string, or anything that represents a state
    # @return
    # boolean
    def in_path(self, state):
        if self.state == state:
            return True
        elif self.parent == None:
            return False 
        else:
            # recursion
            # just like a linked list,
            # check if the asked state exists
            # in your ancestor line
            return self.parent.in_path(state)
  
    def __eq__(self, other):
        if self is None and other is None:
            return True
        elif self is None:
            return False
        elif other is None:
            return False
        else:
            return self.state == other.state and self.parent == other.parent and \
                   self.action == other.action

In [20]:
s = SearchNode(None, "S", None)
a = SearchNode(0, "A", s)
b = SearchNode(1, "B", s)
s1 = SearchNode(0, "S", a)
c = SearchNode(1, "C", a)
d1 = SearchNode(2, "D", a)
s2 = SearchNode(0, "S", b)
d2 = SearchNode(1, "D", b)
e = SearchNode(2, "E", b)
a1 = SearchNode(0, "A", s1)
b1 = SearchNode(1, "B", s1)
a2 = SearchNode(0, "A", c)
f1 = SearchNode(1, "F", c)
a3 = SearchNode(0, "A", d1)
b2 = SearchNode(1, "B", d1)
f2 = SearchNode(2, "F", d1)
h1 = SearchNode(3, "H", d1)
a4 = SearchNode(0, "A", s2)
b3 = SearchNode(1, "B", s2)
a5 = SearchNode(0, "A", d2)
b4 = SearchNode(1, "B", d2)
f3 = SearchNode(2, "F", d2)
h2 = SearchNode(3, "H", d2)
b5 = SearchNode(0, "B", e)
h3 = SearchNode(1, "H", e)

assert s.parent == None
assert a.state == "A" and a.parent == s and a.action == 0
assert b.state == "B" and b.parent == s and b.action == 1
assert h3.state == "H" and h3.parent == e and h3.action == 1
assert a5.path() == [Step(None, "S"), Step(1, "B"), Step(1, "D"), Step(0, "A")]
assert b5.in_path("B")
assert b5.in_path("S")
assert b5.in_path("E")

NameError: name 'Step' is not defined

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
